In [1]:
input_folder = "chemistry"
input_csv = "Synthetic Data Evaluation  - Chemistry.csv" 
output_csv = "Chemistry_output_answers_openai.csv"  

In [ ]:
import pandas as pd
import csv
def generate_prompt(question):
    prompt_template = """
    You are a physics expert tasked with answering questions.
    Answer the following question with a detailed explanation:
    Question: {question}
    provide the final answer in a boxed format like this:
    \n\n\\boxed{{Your final answer here}}\n\n
    """
    return prompt_template.format(question=question)


## Open AI batch code 

In [9]:
import pandas as pd
import openai
import requests
import openai
import json
import os
import csv
from openai import OpenAI
# load dotenv 
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
# Set OpenAI API key
openai_api_key= os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)



def call_openai_api_batch(prompts):
    """
    Call the OpenAI API in batch mode using a hypothetical batch endpoint.
    Adjust the URL and payload as required by the API's documentation.
    """
    # .jsonl file preparation
    batch_requests = []
    for prompt in prompts:
        request_item = {

            "custom_id": f"{prompt['query_id']}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "4.1-mini",
                "messages": [
                    {"role": "user", "content": prompt["prompt"]}
                ],
            }
        }
        batch_requests.append(request_item)

    
    # Write the batch requests to a .jsonl file
    batch_file_path = os.path.join(input_folder, "openai_batch_requests.jsonl")
    with open(batch_file_path, "w") as f:
        for req in batch_requests:
            json_line = json.dumps(req)
            f.write(json_line + "\n")
    batch_input_file = client.files.create(
    file=open(batch_file_path, "rb"),
    purpose="batch"
    )
    print("Batch file uploaded successfully.",batch_input_file)
    batch_input_file_id = batch_input_file.id
    
    response = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "nightly eval job"
    }
    )
    print("Batch job created successfully.",batch_input_file_id)
    print("Batch Object",response)

#### openai batch execution 

In [10]:
# Read the CSV file
input_csv = os.path.join(input_folder, input_csv)
df = pd.read_csv(input_csv)
# use last 5 rows for testing
df = df.tail(5)

FileNotFoundError: [Errno 2] No such file or directory: 'chemistry/chemistry/Synthetic Data Evaluation  - Chemistry.csv'

In [11]:
df

,Task ID,Batch_ID,Domain,Subdomain,Question,Critic Model,Creator Model,Updated Question,Status,Feedback Category,Difficulty Level,Academic Level,Comments,Relevant doc,Relevant topic,Relevant concept
52,SDC153,CHEM_2805_01,Supramolecular Chemistry / Physical Chemistry,Chemical Kinetics,A supramolecular intracellular assembly shows ...,GPT 4.1,4.1-mini,NaN,Selected,NaN,Medium,PG,\documentclass{article}\n\usepackage{amsmath}\...,NaN,NaN,NaN
53,SDC154,CHEM_2805_01,Biophysical Chemistry,Chemical Thermodynamics / Membrane Biology / D...,Calculate the Gibbs free energy change $\Delta...,GPT 4.1,4.1-mini,NaN,Drop,NaN,Easy,UG,This question requires a direct application of...,NaN,NaN,NaN
54,SDC155,CHEM_2805_01,Physical Chemistry,Photochemistry and Spectroscopy,"Upon NIR irradiation at 800 nm, a photoactivat...",GPT 4.1,4.1-mini,NaN,Selected,NaN,Easy,PG,The problem involves a direct application of t...,NaN,NaN,NaN
55,SDC156,CHEM_2805_01,Physical Chemistry,Thermodynamics and Self-Assembly,A synthetic intracellular assembly formed from...,GPT 4.1,4.1-mini,NaN,Selected,NaN,Medium,PG,Requires understanding how to express Gibbs fr...,NaN,NaN,NaN
56,SDC157,CHEM_2805_01,Physical Chemistry / Supramolecular Chemistry,Photochemistry and Radical Kinetics,"During intracellular supramolecular assembly, ...",GPT 4.1,4.1-mini,NaN,Selected,NaN,Easy,PG,Requires the direct application of formulas to...,NaN,NaN,NaN


In [12]:
# Generate prompts for each row and store in a list (preserving order)
prompts = []
for index, row in df.iterrows():
    question = row['Question']
    query_id = row['Task ID']
    # convert to integer
    # query_id = query_id
    prompt = generate_prompt(question)
    prompts.append({"prompt":prompt , "query_id": query_id})


# Call the OpenAI API in batch mode
print("Calling OpenAI batch API...")
call_openai_api_batch(prompts)



Calling OpenAI batch API...
Batch file uploaded successfully. FileObject(id='file-4X6H6YKGHEfo7huwvwKxQJ', bytes=4852, created_at=1748510179, filename='openai_batch_requests.jsonl', object='file', purpose='batch', status='processed', status_details=None, expires_at=None)
Batch job created successfully. file-4X6H6YKGHEfo7huwvwKxQJ
Batch Object Batch(id='batch_683825e4713c8190bb2309939bb38206', completion_window='24h', created_at=1748510180, endpoint='/v1/chat/completions', input_file_id='file-4X6H6YKGHEfo7huwvwKxQJ', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1748596580, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


#### check status of openai batch calling

In [15]:
batch = client.batches.retrieve("batch_683825e4713c8190bb2309939bb38206")
batch

Batch(id='batch_683825e4713c8190bb2309939bb38206', completion_window='24h', created_at=1748510180, endpoint='/v1/chat/completions', input_file_id='file-4X6H6YKGHEfo7huwvwKxQJ', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='model_not_found', line=1, message="The provided model '4.1-mini' is not supported by the Batch API.", param='body.model'), BatchError(code='model_not_found', line=2, message="The provided model '4.1-mini' is not supported by the Batch API.", param='body.model'), BatchError(code='model_not_found', line=3, message="The provided model '4.1-mini' is not supported by the Batch API.", param='body.model'), BatchError(code='model_not_found', line=4, message="The provided model '4.1-mini' is not supported by the Batch API.", param='body.model'), BatchError(code='model_not_found', line=5, message="The provided model '4.1-mini' is not supported by the Batch API.", param='body.m

#### Openai API result 

In [ ]:
file_response = client.files.content("file-GMXw8eAF7vd4NGS9bwg4Dw")
print(file_response.text)


batch_response_file = os.path.join(input_folder, "batch_response.json")
#save the response to a file
with open(batch_response_file, "w") as f:
    f.write(file_response.text) 

{"id": "batch_req_67efdaa6d87881909e349dbc8c9bcebd", "custom_id": "18305", "response": {"status_code": 200, "request_id": "10ff20ede3f99bcd8f1de13e49daca24", "body": {"id": "chatcmpl-BIaxE4cnDD1Vw6APYzxnnKEP4PS2t", "object": "chat.completion", "created": 1743771656, "model": "o1-2024-12-17", "choices": [{"index": 0, "message": {"role": "assistant", "content": "First, let us restate the two given systems more systematically.  We have:\n\n\u2022 System (1):  \n  (1a)  \u1e8b\u2081 \u2212 \u1e8f\u2081 = (x\u2081 \u2212 y\u2081) \u2212 \u03bd\u2081 (cos x\u2081 \u2212 sin x\u2081),  \n  (1b)  \u1e8b\u2081 = 1.\n\n\u2022 System (2):  \n  (2a)  \u1e8b\u2082 \u2212 \u1e8f\u2082 = (x\u2082 \u2212 y\u2082) \u2212 \u03bd\u2082 (sin(\u2212x\u2082) + cos(\u2212x\u2082)),  \n  (2b)  \u1e8b\u2082 = 1.\n\nBecause sin(\u2212\u03b8) = \u2212sin \u03b8 and cos(\u2212\u03b8) = cos \u03b8, we have  \nsin(\u2212x\u2082) + cos(\u2212x\u2082) = \u2212sin x\u2082 + cos x\u2082 = (cos x\u2082 \u2212 sin x\u208

_______________

## getting answers from OpenAI batch response

#### prompt for getting answer from response 

In [48]:
def make_prompt(question,response):
    prompt_template = """
    Given the following question and model response, output only the final answer from the detailed explanation response.
    Question: {question}
    Answer: {response}
    """
    return prompt_template.format(question=question, response=response)

#### preparing batch 

In [ ]:
import json
import csv

results = []

# Read the file line by line
with open(batch_response_file, "r") as file:
    for line in file:
        line = line.strip()
        if not line:  # Skip any empty lines
            continue
        try:
            # Parse each JSON object from the line
            data = json.loads(line)
            
            # Extract custom_id
            custom_id = data.get("custom_id", "")
            
            # Extract content from the first choice in the nested structure
            choices = data.get("response", {}).get("body", {}).get("choices", [])
            if choices:
                content = choices[0].get("message", {}).get("content", "")
            else:
                content = ""
            
            results.append((custom_id, content))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON line: {e}")

# Write the extracted data to a CSV file
batch_response_file_csv = os.path.join(input_folder, "batch_response.csv")
with open(batch_response_file_csv, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    # Write header row
    writer.writerow(["custom_id", "model_response"])
    # Write each extracted row
    writer.writerows(results)

print("Data has been successfully written to batch_response.csv")

# OPEN response.csv
response_df = pd.read_csv(batch_response_file_csv)

# open Copyof [Deccan AI] PCM samples - Mathematics.csv
input_df = pd.read_csv(input_csv)

# Merge the two DataFrames on the 'Query Id' column
merged_df = pd.merge(input_df, response_df, left_on='Query Id', right_on='custom_id', how='inner')

Data has been successfully written to response.csv


In [ ]:
def openai_response_to_answer(prompts):

    # .jsonl file preparation
    batch_requests = []
    for prompt in prompts:
        request_item = {

            "custom_id": f"{prompt["query_id"]}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "o1",
                "messages": [
                    {"role": "user", "content": prompt["prompt"]}
                ],
            }
        }
        batch_requests.append(request_item)

    
    # Write the batch requests to a .jsonl file'
    batch_file_path_response_answer = os.path.join(input_folder, "openai_response_to_answer.jsonl")
    with open(batch_file_path_response_answer, "w") as f:
        for req in batch_requests:
            json_line = json.dumps(req)
            f.write(json_line + "\n")
    batch_input_file = client.files.create(
    file=open(batch_file_path_response_answer, "rb"),
    purpose="batch"
    )
    print("Batch file uploaded successfully.",batch_input_file)
    batch_input_file_id = batch_input_file.id
    
    response = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "nightly eval job"
    }
    )
    print("Batch job created successfully.",batch_input_file_id)
    print("Batch Object",response)

In [ ]:
# Call the OpenAI API in batch mode
prompts_response_to_answer = []
for index, row in merged_df.iterrows():
    question = row['User Query']
    response = row['model_response']
    query_id = row['Query Id']
    # convert to integer
    query_id = int(query_id)
    prompt = make_prompt(question,response)
    prompts_response_to_answer.append({"prompt":prompt , "query_id": query_id})


# Call the OpenAI API in batch mode
print("Calling OpenAI batch API...")
openai_response_to_answer(prompts_response_to_answer)

Calling OpenAI batch API...
Batch file uploaded successfully. FileObject(id='file-62Xh5A6oSX1pvwKwa9RexW', bytes=64759, created_at=1743774003, filename='openai_response_to_answer.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)
Batch job created successfully. file-62Xh5A6oSX1pvwKwa9RexW
Batch Object Batch(id='batch_67efe133f1c0819086866c9eca3e2a0a', completion_window='24h', created_at=1743774003, endpoint='/v1/chat/completions', input_file_id='file-62Xh5A6oSX1pvwKwa9RexW', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1743860403, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


#### check batch status

In [54]:
batch = client.batches.retrieve("batch_67efe133f1c0819086866c9eca3e2a0a")
batch

Batch(id='batch_67efe133f1c0819086866c9eca3e2a0a', completion_window='24h', created_at=1743774003, endpoint='/v1/chat/completions', input_file_id='file-62Xh5A6oSX1pvwKwa9RexW', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1743774097, error_file_id=None, errors=None, expired_at=None, expires_at=1743860403, failed_at=None, finalizing_at=1743774096, in_progress_at=1743774007, metadata={'description': 'nightly eval job'}, output_file_id='file-Gip1eAC3YRgzxjQnCBwDRv', request_counts=BatchRequestCounts(completed=9, failed=0, total=9))

#### save to file with format question,gold resonse, gold answer, openai response, openai answer .

In [ ]:
file_response = client.files.content("file-Gip1eAC3YRgzxjQnCBwDRv")
print(file_response.text)

batch_answer_file = os.path.join(input_folder, "batch_answer.json")
#save the response to a file
with open(batch_answer_file, "w") as f:
    f.write(file_response.text)

{"id": "batch_req_67efe190dc548190ac0c3a20bd796245", "custom_id": "18305", "response": {"status_code": 200, "request_id": "8878475a9232feffc9396b9d9172b5aa", "body": {"id": "chatcmpl-BIbZVOjicIYrUtF7E9FQwXGu3X2PG", "object": "chat.completion", "created": 1743774029, "model": "o1-2024-12-17", "choices": [{"index": 0, "message": {"role": "assistant", "content": "On the synchronization manifold x\u2081 = x\u2082 = x, define z\u2081 := x \u2212 y\u2081 and z\u2082 := x \u2212 y\u2082. Then the reduced (invariant) system in \u211d\u00b3 is:\n\nx\u0307 = 1,  \nz\u0307\u2081 = z\u2081 \u2212 \u03bd\u2081 (cos x \u2212 sin x),  \nz\u0307\u2082 = z\u2082 + \u03bd\u2082 (sin x \u2212 cos x),\n\nwith mismatch parameter \u03bd = \u03bd\u2082 \u2212 \u03bd\u2081 if needed.", "refusal": null, "annotations": []}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 1629, "completion_tokens": 693, "total_tokens": 2322, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}

___________________________________________

## Judge Module

#### prompt for Judge 

In [ ]:
def evaluate_model_answer_prompt(question, gold_answer, model_answer):
    """
    Evaluate the model's answer against the gold answer.
    """
    # Define the prompt for evaluation    
    prompt = f"""
    You are an expert evaluator. Here is the question, the correct (gold) answer, and an candidate answer.
    Based on the gold answer, determine if the candidate answer is correct. 
    Answer only with "Correct" if they match or "Incorrect" if they do not.
    
    Question: {question}
    Gold Answer: {gold_answer}
    Model Answer: {model_answer}
    
    """
    return prompt

#### batch preparation for judge 

In [ ]:
import json
import csv

results = []

# Read the file line by line
with open(batch_answer_file, "r") as file:
    for line in file:
        line = line.strip()
        if not line:  # Skip any empty lines
            continue
        try:
            # Parse each JSON object from the line
            data = json.loads(line)
            
            # Extract custom_id
            custom_id = data.get("custom_id", "")
            
            # Extract content from the first choice in the nested structure
            choices = data.get("response", {}).get("body", {}).get("choices", [])
            if choices:
                content = choices[0].get("message", {}).get("content", "")
            else:
                content = ""
            
            results.append((custom_id, content))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON line: {e}")

# Write the extracted data to a CSV file
batch_answer_file_csv = os.path.join(input_folder, "batch_answer.csv")

with open(batch_answer_file_csv, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    # Write header rowresponse
    writer.writerow(["custom_id", "model_answer"])
    # Write each extracted row
    writer.writerows(results)

print("Data has been successfully written to batch_response_answer.csv")

# OPEN response.csv
response_df = pd.read_csv(batch_answer_file_csv)

# open Copyof [Deccan AI] PCM samples - Mathematics.csv
input_df = pd.read_csv(input_csv)

# Merge the two DataFrames on the 'Query Id' column
merged_df_resonse_to_answer = pd.merge(input_df, response_df, left_on='Query Id', right_on='custom_id', how='inner')

Data has been successfully written to batch_response_answer.csv


In [ ]:

def openai_judge(prompts):

    # .jsonl file preparation
    batch_requests = []
    for prompt in prompts:
        request_item = {

            "custom_id": f"{prompt["query_id"]}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "o1",
                "messages": [
                    {"role": "user", "content": prompt["prompt"]}
                ],
            }
        }
        batch_requests.append(request_item)

    
    # Write the batch requests to a .jsonl file

    batch_file_path_judge = os.path.join(input_folder, "openai_judge.jsonl")
    with open(batch_file_path_judge, "w") as f:
        for req in batch_requests:
            json_line = json.dumps(req)
            f.write(json_line + "\n")
    batch_input_file = client.files.create(
    file=open(batch_file_path_judge, "rb"),
    purpose="batch"
    )
    print("Batch file uploaded successfully.",batch_input_file)
    batch_input_file_id = batch_input_file.id
    
    response = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "nightly eval job"
    }
    )
    print("Batch job created successfully.",batch_input_file_id)
    print("Batch Object",response)

In [66]:
prompts_judge = []
for index, row in merged_df_resonse_to_answer.iterrows():
    question = row['User Query']
    gold_answer = row['Unnamed: 5']
    model_answer = row['model_answer']
    query_id = row['Query Id']
    # convert to integer
    query_id = int(query_id)
    prompt = evaluate_model_answer_prompt(question,gold_answer,model_answer)
    prompts_judge.append({"prompt":prompt , "query_id": query_id})

# Call the OpenAI API in batch mode
print("Calling OpenAI batch API...")
openai_judge(prompts_judge)


Calling OpenAI batch API...
Batch file uploaded successfully. FileObject(id='file-FkweLLgTTk6XNFbd8Bd9Ln', bytes=13185, created_at=1743775499, filename='openai_judge.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)
Batch job created successfully. file-FkweLLgTTk6XNFbd8Bd9Ln
Batch Object Batch(id='batch_67efe70ccf1481908328939c7687a742', completion_window='24h', created_at=1743775500, endpoint='/v1/chat/completions', input_file_id='file-FkweLLgTTk6XNFbd8Bd9Ln', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1743861900, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [73]:
prompts_judge

[{'prompt': '\n    You are an expert evaluator. Here is the question, the correct (gold) answer, and a model\'s answer.\n    Based on the gold answer, determine if the model answer is correct. \n    Answer only with "Correct" if they match or "Incorrect" if they do not.\n\n    Question: Consider the following systems $$ \\dot{x_{1}} - \\dot{y_{1}} = (x_{1} - y_{1}) - \\nu_{1} (\\cos{x_{1}} - \\sin{x_{1}}), ~~~~\\text{with} \\quad \\dot{x_{1}} = 1 \\quad(1) $$ and $$ \\dot{x_{2}} - \\dot{y_{2}} = (x_{2} - y_{2}) - \\nu_{2} (\\sin{(-x_{2})} + \\cos{(-x_{2})}), \\text{with} \\quad \\dot{x_{2}} = 1 \\quad(2) $$ Now consider a scenario where the $x_{1,2}$- components from both systems synchronize whenever system (1) sends signals to the system (2) through the linear coupling term in the form $f(x_{2} - x_{1})$ and coupling strength $\\Psi$. If $\\nu = \\nu_2 - \\nu_{1}$ measures the order of mismatch, then what are the equations of the invariant manifold for systems (1) and (2) in the ${R}^

#### check batch status judge 

In [70]:
batch = client.batches.retrieve("batch_67efe70ccf1481908328939c7687a742")
batch

Batch(id='batch_67efe70ccf1481908328939c7687a742', completion_window='24h', created_at=1743775500, endpoint='/v1/chat/completions', input_file_id='file-FkweLLgTTk6XNFbd8Bd9Ln', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1743775714, error_file_id=None, errors=None, expired_at=None, expires_at=1743861900, failed_at=None, finalizing_at=1743775713, in_progress_at=1743775501, metadata={'description': 'nightly eval job'}, output_file_id='file-3nPbKhnQBjCyQDxmHtTGSb', request_counts=BatchRequestCounts(completed=9, failed=0, total=9))

#### save to output will binary correct-incorrect

In [ ]:
file_response = client.files.content("file-3nPbKhnQBjCyQDxmHtTGSb")
print(file_response.text)
#save the response to a file
batch_judge_file = os.path.join(input_folder, "batch_judge.json")
with open(batch_judge_file, "w") as f:
    f.write(file_response.text)

{"id": "batch_req_67efe7e147508190924fa01536ba2d8e", "custom_id": "18305", "response": {"status_code": 200, "request_id": "b60f6f4e34241a7c3184efc77f9fce27", "body": {"id": "chatcmpl-BIbxNDMxWk2uORm51DZ9RUC8LRYWQ", "object": "chat.completion", "created": 1743775509, "model": "o1-2024-12-17", "choices": [{"index": 0, "message": {"role": "assistant", "content": "Incorrect", "refusal": null, "annotations": []}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 492, "completion_tokens": 1933, "total_tokens": 2425, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 1920, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "system_fingerprint": "fp_688960522e"}}, "error": null}
{"id": "batch_req_67efe7e15bb08190815142fc7ac05ff0", "custom_id": "18391", "response": {"status_code": 200, "request_id": "9c0d54d8285cc09120e080b4d5759a5e", "body": {"id

In [9]:
# read the batch_response_judge.json file and write in a csv file with custom_id and judgement

import json
import csv
results = []
# Read the file line by line

with open(batch_judge_file, "r") as file:
    for line in file:
        line = line.strip()
        if not line:  # Skip any empty lines
            continue
        try:
            # Parse each JSON object from the line
            data = json.loads(line)
            
            # Extract custom_id
            custom_id = data.get("custom_id", "")
            
            # Extract content from the first choice in the nested structure
            choices = data.get("response", {}).get("body", {}).get("choices", [])
            if choices:
                content = choices[0].get("message", {}).get("content", "")
            else:
                content = ""
            
            results.append((custom_id, content))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON line: {e}")
# Write the extracted data to a CSV file
batch_judge_file_csv = os.path.join(input_folder, "batch_response_judge.csv")
with open(batch_judge_file_csv, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    # Write header row
    writer.writerow(["custom_id", "judgement"])
    # Write each extracted row
    writer.writerows(results)
print("Data has been successfully written to batch_response_judge.csv")

Data has been successfully written to batch_response_judge.csv


# Compile everything in 1 file 

In [ ]:
# read from batch_response_answer.csv , response.csv , batch_response_judge.csv , Copy of [Deccan AI] PCM samples - Mathematics.csv
# combine all the dataframes using custom_id
import pandas as pd
input_df = pd.read_csv(input_csv)
# keep only the Query Id column, domain, subdomain, and User Query
# keep the user response (ground truth) and keep the 6th column whihch is the gold answer 
input_df = input_df[['Query Id', 'Domain', 'Sub Domain', 'User Query', 'Unnamed: 5','User Response (Ground Truth)']]
# rename unnamed: 5 to gold_answer 
input_df = input_df.rename(columns={'Unnamed: 5': 'Gold_answer', 'User Response (Ground Truth)': 'Gold_response'})

response_df = pd.read_csv(batch_response_file_csv)
answer_df = pd.read_csv(    batch_answer_file_csv)
judge_df = pd.read_csv(batch_judge_file_csv)
# Merge the dataframes on custom_id
merged_df = pd.merge(input_df, response_df, left_on='Query Id', right_on='custom_id', how='inner')
merged_df = pd.merge(merged_df, answer_df, left_on='Query Id', right_on='custom_id', how='inner')
merged_df = pd.merge(merged_df, judge_df, left_on='Query Id', right_on='custom_id', how='inner')

# drop the custom_id column
merged_df = merged_df.drop(columns=['custom_id_x', 'custom_id_y', 'custom_id'])

In [ ]:
merged_df
# save the merged dataframe to a csv file
final_output_file = os.path.join(input_folder, "final_output.csv")
merged_df.to_csv(final_output_file, index=False)

______________

## Judge the Response Module 

In [ ]:
def call_gemini_api_batch(prompts):
    """
    Call the Gemini API in batch mode using a hypothetical batch endpoint.
    Adjust the URL, headers, and payload according to the Gemini API documentation.
    """
        # .jsonl file preparation
    batch_requests = []
    for i, prompt in enumerate(prompts):
        request_item = {
            "request": {
                "contents":
                    [
                        {
                            "role": "user",
                            "parts":[
                                {
                                    "text": prompt
                                }
                            ]
                        }
                    ]
                
            }
        }
        batch_requests.append(request_item)

    # Write the batch requests to a .jsonl file
    batch_file_path = "gemini_batch_requests.jsonl"
    with open(batch_file_path, "w") as f:
        for req in batch_requests:
            json_line = json.dumps(req)
            f.write(json_line + "\n")

    import time

    from google import genai

    client = genai.Client(api_key='my-api-key')


    job = client.batches.create(
    model='gemini-1.5-flash-002',
    src='bq://my-project.my-dataset.my-table',
    )
    print("Batch job created successfully.",job)

    

    


In [ ]:
def claude_batch(prompts):
    """
    Call the Claude API in batch mode using a hypothetical batch endpoint.
    Adjust the URL, headers, and payload according to the Claude API documentation.
    """
    import anthropic
    from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
    from anthropic.types.messages.batch_create_params import Request

    client = anthropic.Anthropic(api_key="YOUR_CLAUDE_API_KEY")
    # .jsonl file preparation
    batch_requests = []
    for i, prompt in enumerate(prompts):
        request_item = Request(
            custom_id=f"request-{i+1}",
            params=MessageCreateParamsNonStreaming(
                model="claude-3-7-sonnet-20250219",
                messages=[
                {
                    "role": "user", 
                    "content": prompt
                }]
            )

        )
        batch_requests.append(request_item)

    # # Write the batch requests to a .jsonl file
    # batch_file_path = "claude_batch_requests.jsonl"
    # with open(batch_file_path, "w") as f:
    #     for req in batch_requests:
    #         json_line = json.dumps(req)
    #         f.write(json_line + "\n")

    message_batch = client.messages.batches.create(
        requests=batch_requests
    )
    print("Batch job created successfully.",message_batch)

In [ ]:

# -------------------------------
# Single API Call Function
# -------------------------------

from openai import OpenAI

deepseek_client = OpenAI(api_key="<DeepSeek API Key>", base_url="https://api.deepseek.com")
def call_deepseek_api(prompt):
    """
    Call the Deepseek API for a single prompt.
    Adjust the URL, headers, and payload according to Deepseek API documentation.
    """
    try:
        response = deepseek_client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "user", "content": prompt}
            ],
        )
        answer = response['choices'][0]['message']['content']
    except Exception as e:
        answer = f"Error: {str(e)}"
    return answer

def together_api_call(prompt):
    """
    Call the Together API for a single prompt.
    
    """
    from together import Together

    client = Together()

    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1",
        messages=[{"role": "user", "content": prompt}],
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content
    

# -------------------------------
# Process CSV and Record Answers
# -------------------------------
def process_questions(input_csv, output_csv):
    """
    Read the CSV file, generate prompts for each question, batch call OpenAI and Gemini APIs,
    call Deepseek API for each prompt individually, and record all answers in new columns.
    """
    df = pd.read_csv(input_csv)
    df['deepseek_answer'] = ""
    
    # Generate prompts for each row and store in a list (preserving order)
    prompts = []
    for index, row in df.iterrows():
        question = row['User Query']
        prompt = generate_prompt(question)
        prompts.append(prompt)

    
    # Single prompt calling for Deepseek (or other models that do not support batch calls)
    deepseek_answers = []
    print("Calling Deepseek API for each prompt...")
    for prompt in prompts:
        ans = call_deepseek_api(prompt)
        deepseek_answers.append(ans)
    
    df['deepseek_answer'] = deepseek_answers
    
    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

# -------------------------------
# Main Execution
# -------------------------------



In [19]:
input_csv = "Copy of [Deccan AI] PCM samples - Mathematics.csv"   # Replace with your input CSV file path
output_csv = "output_answers.csv"     # Replace with your desired output CSV file path

df = pd.read_csv(input_csv)


# Generate prompts for each row and store in a list (preserving order)
prompts = []
for index, row in df.iterrows():
    question = row['User Query']
    prompt = generate_prompt(question)
    prompts.append(prompt)

# Call the OpenAI API in batch mode
print("Calling OpenAI batch API...")
call_openai_api_batch(prompts)
# print("Calling Gemini batch API...")
# call_gemini_api_batch(prompts)




Calling OpenAI batch API...
Batch file uploaded successfully. FileObject(id='file-5ZrLE12oNxSULCKWfCMb6p', bytes=6862, created_at=1743592383, filename='openai_batch_requests.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)
Batch job created successfully. file-5ZrLE12oNxSULCKWfCMb6p
Batch Object Batch(id='batch_67ed1bc062848190934e090d59bf7491', completion_window='24h', created_at=1743592384, endpoint='/v1/chat/completions', input_file_id='file-5ZrLE12oNxSULCKWfCMb6p', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1743678784, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [16]:
# from openai import OpenAI
# client = OpenAI()

batch = client.batches.retrieve("file-SkjSkiYhkEuFfTSkVi3Rxs")
print(batch)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'batch_id': 'file-SkjSkiYhkEuFfTSkVi3Rxs'. Expected an ID that begins with 'batch'.", 'type': 'invalid_request_error', 'param': 'batch_id', 'code': 'invalid_value'}}

# Evaluation 

In [ ]:
import pandas as pd
import openai

# Set your OpenAI API key
openai.api_key = 'YOUR_API_KEY_HERE'

def judge_answer(question, gold_answer, model_answer):
    """
    Constructs a prompt to have OpenAI judge whether the model_answer matches the gold_answer.
    Returns "Correct" if they match, otherwise "Incorrect".
    """
    prompt = f"""
    You are an expert evaluator. Here is the question, the correct (gold) answer, and a model's answer.
    
    Question: {question}
    Gold Answer: {gold_answer}
    Model Answer: {model_answer}
    
    Based on the gold answer, determine if the model answer is correct. 
    Answer only with "Correct" if they match or "Incorrect" if they do not.
    """
    
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=10,  # Small output since we expect a binary answer
        temperature=0  # Deterministic output
    )
    
    return response.choices[0].text.strip()

# Load your CSV file (update the path as needed)
df = pd.read_csv("your_file.csv")

# Apply the judging function to each row for both Gemini and Deepseek responses
df['gemini_judgment'] = df.apply(
    lambda row: judge_answer(row['question'], row['gold label'], row['gemini_response']), axis=1
)
df['deepseek_judgment'] = df.apply(
    lambda row: judge_answer(row['question'], row['gold label'], row['deepseek response']), axis=1
)

# Save the updated DataFrame with judgments to a new CSV file
df.to_csv("judged_results.csv", index=False)

print("Judgment complete. Results saved to judged_results.csv.")


### Misc

In [24]:
def openai_response_to_answer(prompts):

    # .jsonl file preparation
    batch_requests = []
    for prompt in prompts:
        request_item = {

            "custom_id": f"{prompt["query_id"]}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "o1",
                "messages": [
                    {"role": "user", "content": prompt["prompt"]}
                ],
            }
        }
        batch_requests.append(request_item)

    
    # Write the batch requests to a .jsonl file'
    batch_file_path_response_answer = os.path.join(input_folder, "user_response_to_answer.jsonl")
    with open(batch_file_path_response_answer, "w") as f:
        for req in batch_requests:
            json_line = json.dumps(req)
            f.write(json_line + "\n")
    
    batch_input_file = client.files.create(
    file=open(batch_file_path_response_answer, "rb"),
    purpose="batch"
    )
    print("Batch file uploaded successfully.",batch_input_file)
    batch_input_file_id = batch_input_file.id
    
    response = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "nightly eval job"
    }
    )
    print("Batch job created successfully.",batch_input_file_id)
    print("Batch Object",response)

In [27]:
file = "Vega 1.0 - 50 Sample - Sheet1.csv"
input_folder = "50_sample"
file = os.path.join(input_folder, file)
df = pd.read_csv(file)
df = df.dropna(subset=['Sub Task Id'])
df = df.dropna(subset=['User Query'])
def make_prompt(question,response):
    prompt_template = """
    Given the following question and response, output only the final answer from the detailed explanation response.
    Question: {question}
    Answer: {response}
    """
    return prompt_template.format(question=question, response=response)
# Call the OpenAI API in batch mode
prompts_response_to_answer = []
for index, row in df.iterrows():
    question = row['User Query']
    response = row['User Response']
    query_id = row['Sub Task Id']
    # convert to integer
    query_id = int(query_id)
    prompt = make_prompt(question,response)
    prompts_response_to_answer.append({"prompt":prompt , "query_id": query_id})

    
openai_response_to_answer(prompts_response_to_answer)


Batch file uploaded successfully. FileObject(id='file-BhsvoJkdukQRUBTv91DHiF', bytes=54188, created_at=1744011889, filename='user_response_to_answer.jsonl', object='file', purpose='batch', status='processed', expires_at=None, status_details=None)
Batch job created successfully. file-BhsvoJkdukQRUBTv91DHiF
Batch Object Batch(id='batch_67f38271f5048190aa85fe5fbc571814', completion_window='24h', created_at=1744011889, endpoint='/v1/chat/completions', input_file_id='file-BhsvoJkdukQRUBTv91DHiF', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1744098289, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [28]:
batch = client.batches.retrieve("batch_67f38271f5048190aa85fe5fbc571814")
batch

Batch(id='batch_67f38271f5048190aa85fe5fbc571814', completion_window='24h', created_at=1744011889, endpoint='/v1/chat/completions', input_file_id='file-BhsvoJkdukQRUBTv91DHiF', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1744011956, error_file_id=None, errors=None, expired_at=None, expires_at=1744098289, failed_at=None, finalizing_at=1744011955, in_progress_at=1744011890, metadata={'description': 'nightly eval job'}, output_file_id='file-E4cmZeoRV4ZwE2RAyUBPdv', request_counts=BatchRequestCounts(completed=20, failed=0, total=20))

In [40]:
file_response = client.files.content("file-E4cmZeoRV4ZwE2RAyUBPdv")
print(file_response.text)




# parse the response and save to a csv file
import json
import csv

results = []
# Read the file line by line
for line in file_response.text.splitlines():
    # for line in file:
        line = line.strip()
        if not line:  # Skip any empty lines
            continue
        try:
            # Parse each JSON object from the line
            data = json.loads(line)
            
            # Extract custom_id
            custom_id = data.get("custom_id", "")
            
            # Extract content from the first choice in the nested structure
            choices = data.get("response", {}).get("body", {}).get("choices", [])
            if choices:
                content = choices[0].get("message", {}).get("content", "")
            else:
                content = ""
            
            results.append((custom_id, content))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON line: {e}")


df = pd.DataFrame(results, columns=["custom_id", "answer"])
df2 = pd.read_csv(file)
df['custom_id'] = df['custom_id'].astype(str)
df2['Sub Task Id'] = df2['Sub Task Id'].astype(str)

# Merge the two DataFrames on the 'Query Id' column
merged_df = pd.merge(df, df2, left_on='custom_id', right_on='Sub Task Id', how='inner')
# drop the custom_id column
merged_df = merged_df.drop(columns=['custom_id'])

# save the merged dataframe to a csv file
final_output_file = os.path.join(input_folder, "Vega_with_answer_50.csv")
merged_df.to_csv(final_output_file, index=False)
print("Merged DataFrame saved to", final_output_file)

{"id": "batch_req_67f382b389b08190a40bd04a6d2b2e55", "custom_id": "18290", "response": {"status_code": 200, "request_id": "8aabdacaca4306744fa6bda85e453b13", "body": {"id": "chatcmpl-BJbSxqjquCK21rNnl9990QKPhhMkB", "object": "chat.completion", "created": 1744011951, "model": "o1-2024-12-17", "choices": [{"index": 0, "message": {"role": "assistant", "content": "The possible values of k are 1/2 and -1.", "refusal": null, "annotations": []}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 480, "completion_tokens": 217, "total_tokens": 697, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 192, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "system_fingerprint": "fp_688960522e"}}, "error": null}
{"id": "batch_req_67f382b39960819088d95e8688cc18a6", "custom_id": "18292", "response": {"status_code": 200, "request_id": "678da1ee88fc2a3401c

In [39]:
merged_df

,answer,Sub Task Id,User Query,User Response,o3-mini Response,o1-preview Response,DOMAIN
0,The possible values of k are 1/2 and -1.,18290,Consider the given equation: $\frac{a}{b+c} = ...,We have $$\frac{a}{b+c} = \frac{b}{c+a} = \fra...,We start with the equation\n\n a/(b + c) = b/...,To determine all possible real values of \( k ...,Algebra
1,1024,18292,"Given that the real numbers $( x, y, z, w )$ s...",### Step 1: Relating Power Sums to Elementary ...,Let the four numbers be viewed as the roots of...,"To solve this problem, we can use Newton's ide...",Algebra
2,10,18320,Consider the quadratic function $f(x) = ax^2 +...,## Step 1: Recalling the Vertex Form of a Quad...,We start by trying to build a quadratic of the...,"To solve this problem, we'll start by modeling...",Algebra
3,x = 3/2 or x = -1.,18473,Find all the values of $x$ that satisfy the fo...,### Step 1: Simplifying the determinant using ...,We wish to solve\n\n Determinant = \n | x³...,To solve the determinantal equation:\n\n\[\n\b...,Algebra
4,"The arithmetic relationship that a, b, and c m...",18475,"In a classroom, three students $a$, $b$, and $...",Given the equation: \n \n$$ \n(x-a)(x-b) + ...,We begin with the equation\n\n (x – a)(x – b)...,"To solve this problem, we need to determine th...",Algebra
5,x = (2√5 - 1) / 2,18476,**Solve for real $x$ in:** $$ \nx + \sqrt{2x ...,**Step 1: Defining the Substitution** \nLet ...,We wish to solve\n x + √(2x – 1) = √(x + √(2x...,To solve the equation:\n\n\[ x + \sqrt{2x - 1}...,Algebra
6,18,18485,"Consider the subspace of $\mathbb{R}^{20}$, $$...","To find the dimension of $W_1 + W_2$, we analy...",We start by understanding the two subspaces.\n...,"To solve this problem, we need to find the dim...",Algebra
7,The subgroup of GLₙ(ℝ) consisting of matrices ...,18491,What is the structure of the subgroup of $GL_n...,### Step 1: Eigenvalue Condition\n\nA matrix $...,We will show that every invertible real matrix...,The subgroup \( G \) of \( GL_n(\mathbb{R}) \)...,Algebra
8,35,18495,"Suppose $x, y$ and $z$ are non-zero real numbe...",### Step 1: Simplify the Expression\n We start...,"We start with the expression\n\n F(x, y, z) =...",To find the minimum value of the expression\n\...,Algebra
9,120600,18500,"Let, $$S_m=\sum_{r=1}^{4m}(-1)^\frac{r(r+1)}{2...",### Step 1: Analyzing the Sign Sequence \n \...,We are asked to compute\n\n Sₘ = ∑₍r₌₁₎^(4m) ...,"To solve for \( S_{50} \), we need to calculat...",Algebra
